In [10]:
# In this version of OGA with random dictionaries, we use QMC to evaluate the loss function. 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_dtype(torch.float64)


class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1

def generate_relu_dict4D(N_list):
    
    N = np.prod(N_list) 

    grid_indices = [np.linspace(0,1,N_item,endpoint=False) for N_item in N_list]
    grid = np.meshgrid(*grid_indices,indexing='ij')
    grid_coordinates = np.column_stack([axis.ravel() for axis in grid]) 
    samples = torch.tensor(grid_coordinates) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]]) # 2 * sqrt(d)
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)
    f5 = torch.zeros(N,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor.to(device)


def generate_relu_dict4D_QMC(s,N0):
    # Sob = torch.quasirandom.SobolEngine(dimension =4, scramble= True, seed=None) 
    # samples = Sob.draw(N0).double() 

    # for i in range(s-1):
    #     samples = torch.cat([samples,Sob.draw(N0).double()],0)

    # Monte Carlo 
    samples = torch.rand(s*N0,4) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]])
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)
    f5 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor.to(device)

def adjust_neuron_position(my_model, dims = 3):

    def create_mesh_grid(dims, pts):
        mesh = torch.tensor(list(itertools.product(pts,repeat=dims)))
        vertices = mesh.reshape(len(pts) ** dims, -1) 
        return vertices
    counter = 0 
    # positions = torch.tensor([[0.,0.],[0.,1.],[1.,1.],[1.,0.]])
    pts = torch.tensor([0.,1.])
    positions = create_mesh_grid(dims,pts) 
    neuron_num = my_model.fc1.bias.size(0)
    for i in range(neuron_num): 
        w = my_model.fc1.weight.data[i:i+1,:]
        b = my_model.fc1.bias.data[i]
    #     print(w,b)
        values = torch.matmul(positions,w.T) # + b
        left_end = - torch.max(values)
        right_end = - torch.min(values)
        offset = (right_end - left_end)/50
        if b <= left_end + offset/2 : 
            b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
            my_model.fc1.bias.data[i] = b 
        if b >= right_end - offset/2 :
            if counter < (dims+1):
#                 print("here")
                counter += 1
            else: # (d + 1) or more 
                b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
                my_model.fc1.bias.data[i] = b 
    return my_model

def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 



def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct"):
    """
    calls the following functions (dependency): 
    1. GQ_piecewise_2D
    input: the nn model containing parameter 
    1. define the loss function  
    2. take derivative to extract the linear system A
    3. call the cg solver in scipy to solve the linear system 
    output: sol. solution of Ax = b
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
    weighted_basis_value_col = basis_value_col * weights 
    jac = weighted_basis_value_col.t() @ basis_value_col 
     
    rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 

def OGAL2FittingReLU4D(my_model,target,N_list,num_epochs, M, k =1, linear_solver = "direct",num_batches = 1): 
    
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 
    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    # samples for QMC integral
    start_time = time.time()
    # Sob_integral = torch.quasirandom.SobolEngine(dimension =4, scramble= False, seed=None) 
    # integration_points = Sob_integral.draw(M).double() 
    # integration_points = integration_points.to(device)
    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = 4) 
    print("generate sob sequence:", time.time() - start_time) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    N = np.prod(N_list) 
    relu_dict_parameters = generate_relu_dict4D(N_list).t() 
#     num_batches = 2**3 #8, divide dictionary elements 
    batch_size = N//num_batches # batch_size * i : batch_size * (i+1), i = 0,..., num_batches - 1 
    
    for i in range(num_epochs): 
  
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            with torch.no_grad(): 
                func_values = target(integration_points) - my_model(integration_points)

        start_time = time.time() 

        output = torch.zeros(N,1)
        
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:4, j:end_index] ) - relu_dict_parameters[4, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
#         basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting    
#         output = torch.abs(torch.matmul(basis_values,func_values))/M # 
#         neuron_index = torch.argmax(output.flatten())
        
        print("argmax time taken, ", time.time() - start_time)
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[0:4, neuron_index]) # 
        list_b.append(-relu_dict_parameters[4,neuron_index])
        num_neuron += 1
        my_model = model(4,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        #Todo 
        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("time taken: ",time.time() - all_start_time)
    return err, my_model


def OGAL2FittingReLU4D_QMC(my_model,target,s,N0,num_epochs, M, k =1, linear_solver = "direct", num_batches = 1): 
    
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    # samples for QMC integral
    start_time = time.time()
    # Sob_integral = torch.quasirandom.SobolEngine(dimension =4, scramble= False, seed=None) 
    # integration_points = Sob_integral.draw(M).double() 
    # integration_points = integration_points.to(device)
    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = 4) 
    print("generate sob sequence:", time.time() - start_time) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict4D_QMC(s,N0).t()   
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            func_values = target(integration_points) - my_model(integration_points).detach()

        start_time = time.time() 
        
        N = s*N0 
        output = torch.zeros(N,1)
        batch_size = N//num_batches
        
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:4, j:end_index] ) - relu_dict_parameters[4, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
#         basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting
#         output = torch.abs(torch.matmul(basis_values,func_values))/M # 
#         neuron_index = torch.argmax(output.flatten())
        print("argmax time taken, ", time.time() - start_time)
        
        # print(neuron_index)
#         list_w.append(relu_dict_parameters[neuron_index,0:4]) # 
#         list_b.append(-relu_dict_parameters[neuron_index,4])
        list_w.append(relu_dict_parameters[0:4, neuron_index]) # 
        list_b.append(-relu_dict_parameters[4,neuron_index])
        num_neuron += 1
        my_model = model(4,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        #Todo 
        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        func_values = target(integration_points) - my_model(integration_points).detach()
        func_values_sqrd = func_values*func_values

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("total duration: ",time.time() - all_start_time)
    return err, my_model

def print_convergence_order(err, neuron_num_exponent): 

    neuron_nums = [2**j for j in range(2,neuron_num_exponent)]
    err_list = [err[i] for i in neuron_nums ] 

    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))



## Deterministic dictionary 

In [7]:
if __name__ == "__main__": 
    def target(x):
        return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) *torch.sin(pi*x[:,3:4])

    experiment_label = "ex1"
    for k in [4]: 

        for N_list in [[2**4, 2**4, 2**3, 2**3]]: 
            N = np.prod(N_list) 
            print()
            print() 
            exponent = 9  
            num_epochs=  2**exponent 
            M = 2**19 # around 50w 
            print(M)
            my_model = None 
            err, my_model = OGAL2FittingReLU4D(my_model,target,N_list,num_epochs, M, k = k, linear_solver = "ls", num_batches= 2**3)

            filename = experiment_label + "_err_deterministic_Dict_relu_{}_size_{}_num_neurons_{}.pt".format(k,N,num_epochs)
            torch.save(err,filename)
            filename = experiment_label + "_model_deterministic_Dict_relu_{}_size_{}_num_neurons_{}.pt".format(k,N,num_epochs)
            torch.save(my_model.state_dict(),filename) 
    
            print_convergence_order(err,exponent+1) 




524288
generate sob sequence: 0.0068280696868896484
using linear solver:  ls
epoch:  1	argmax time taken,  0.42433714866638184
assembling the matrix time taken:  0.000244140625
solving Ax = b time taken:  0.0001556873321533203
		 time taken minimize linear layer:  0.0004296302795410156
current error:  tensor(0.2000)
epoch:  2	argmax time taken,  0.40897488594055176
assembling the matrix time taken:  0.0002675056457519531
solving Ax = b time taken:  0.0002803802490234375
		 time taken minimize linear layer:  0.0005793571472167969
current error:  tensor(0.1988)
epoch:  3	argmax time taken,  0.40905094146728516
assembling the matrix time taken:  0.0002484321594238281
solving Ax = b time taken:  0.0003077983856201172
		 time taken minimize linear layer:  0.0005877017974853516
current error:  tensor(0.1954)
epoch:  4	argmax time taken,  0.4091217517852783
assembling the matrix time taken:  0.0002639293670654297
solving Ax = b time taken:  0.0003311634063720703
		 time taken minimize linea

epoch:  34	argmax time taken,  0.4111897945404053
assembling the matrix time taken:  0.00026535987854003906
solving Ax = b time taken:  0.0015125274658203125
		 time taken minimize linear layer:  0.0018105506896972656
current error:  tensor(0.0972)
epoch:  35	argmax time taken,  0.41114139556884766
assembling the matrix time taken:  0.000255584716796875
solving Ax = b time taken:  0.0015745162963867188
		 time taken minimize linear layer:  0.0018625259399414062
current error:  tensor(0.0925)
epoch:  36	argmax time taken,  0.4112398624420166
assembling the matrix time taken:  0.0002639293670654297
solving Ax = b time taken:  0.0015597343444824219
		 time taken minimize linear layer:  0.0018563270568847656
current error:  tensor(0.0885)
epoch:  37	argmax time taken,  0.41121411323547363
assembling the matrix time taken:  0.000255584716796875
solving Ax = b time taken:  0.0016422271728515625
		 time taken minimize linear layer:  0.0019309520721435547
current error:  tensor(0.0849)
epoch: 

epoch:  101	argmax time taken,  0.4125101566314697
assembling the matrix time taken:  0.00024890899658203125
solving Ax = b time taken:  0.0051348209381103516
		 time taken minimize linear layer:  0.005417346954345703
current error:  tensor(0.0170)
epoch:  102	argmax time taken,  0.41255974769592285
assembling the matrix time taken:  0.00026297569274902344
solving Ax = b time taken:  0.0053882598876953125
		 time taken minimize linear layer:  0.005684614181518555
current error:  tensor(0.0167)
epoch:  103	argmax time taken,  0.41426873207092285
assembling the matrix time taken:  0.00024771690368652344
solving Ax = b time taken:  0.005248546600341797
		 time taken minimize linear layer:  0.005530118942260742
current error:  tensor(0.0165)
epoch:  104	argmax time taken,  0.414198637008667
assembling the matrix time taken:  0.00026535987854003906
solving Ax = b time taken:  0.005163908004760742
		 time taken minimize linear layer:  0.0054628849029541016
current error:  tensor(0.0163)
epoc

epoch:  135	argmax time taken,  0.41480040550231934
assembling the matrix time taken:  0.0002524852752685547
solving Ax = b time taken:  0.008024454116821289
		 time taken minimize linear layer:  0.008310556411743164
current error:  tensor(0.0105)
epoch:  136	argmax time taken,  0.4149892330169678
assembling the matrix time taken:  0.00026416778564453125
solving Ax = b time taken:  0.008097171783447266
		 time taken minimize linear layer:  0.008394479751586914
current error:  tensor(0.0105)
epoch:  137	argmax time taken,  0.4147634506225586
assembling the matrix time taken:  0.00025343894958496094
solving Ax = b time taken:  0.008182287216186523
		 time taken minimize linear layer:  0.008469581604003906
current error:  tensor(0.0103)
epoch:  138	argmax time taken,  0.41480064392089844
assembling the matrix time taken:  0.0002663135528564453
solving Ax = b time taken:  0.008485794067382812
		 time taken minimize linear layer:  0.008785724639892578
current error:  tensor(0.0103)
epoch:  

epoch:  169	argmax time taken,  0.41551995277404785
assembling the matrix time taken:  0.0002493858337402344
solving Ax = b time taken:  0.009989261627197266
		 time taken minimize linear layer:  0.010272026062011719
current error:  tensor(0.0078)
epoch:  170	argmax time taken,  0.41556835174560547
assembling the matrix time taken:  0.0002505779266357422
solving Ax = b time taken:  0.009911537170410156
		 time taken minimize linear layer:  0.010195493698120117
current error:  tensor(0.0078)
epoch:  171	argmax time taken,  0.41544651985168457
assembling the matrix time taken:  0.0002551078796386719
solving Ax = b time taken:  0.010051727294921875
		 time taken minimize linear layer:  0.010340452194213867
current error:  tensor(0.0076)
epoch:  172	argmax time taken,  0.41561269760131836
assembling the matrix time taken:  0.0002491474151611328
solving Ax = b time taken:  0.009946584701538086
		 time taken minimize linear layer:  0.010229825973510742
current error:  tensor(0.0075)
epoch:  

...

epoch:  373	argmax time taken,  0.41935038566589355
assembling the matrix time taken:  0.00025153160095214844
solving Ax = b time taken:  0.03182268142700195
		 time taken minimize linear layer:  0.03210854530334473
current error:  tensor(0.0050)
epoch:  374	argmax time taken,  0.4193284511566162
assembling the matrix time taken:  0.0002562999725341797
solving Ax = b time taken:  0.031752586364746094
		 time taken minimize linear layer:  0.0320432186126709
current error:  tensor(0.0050)
epoch:  375	argmax time taken,  0.4193432331085205
assembling the matrix time taken:  0.0002512931823730469
solving Ax = b time taken:  0.0320277214050293
		 time taken minimize linear layer:  0.03231334686279297
current error:  tensor(0.0049)
epoch:  376	argmax time taken,  0.4192945957183838
assembling the matrix time taken:  0.0002567768096923828
solving Ax = b time taken:  0.031569480895996094
		 time taken minimize linear layer:  0.0318601131439209
current error:  tensor(0.0050)
epoch:  377	argmax 

epoch:  407	argmax time taken,  0.4210999011993408
assembling the matrix time taken:  0.00024962425231933594
solving Ax = b time taken:  0.03603649139404297
		 time taken minimize linear layer:  0.03632092475891113
current error:  tensor(0.0053)
epoch:  408	argmax time taken,  0.42000555992126465
assembling the matrix time taken:  0.0002579689025878906
solving Ax = b time taken:  0.036832571029663086
		 time taken minimize linear layer:  0.0371246337890625
current error:  tensor(0.0051)
epoch:  409	argmax time taken,  0.42116498947143555
assembling the matrix time taken:  0.0002465248107910156
solving Ax = b time taken:  0.036306142807006836
		 time taken minimize linear layer:  0.03658747673034668
current error:  tensor(0.0053)
epoch:  410	argmax time taken,  0.41999220848083496
assembling the matrix time taken:  0.0002567768096923828
solving Ax = b time taken:  0.03587651252746582
		 time taken minimize linear layer:  0.03616786003112793
current error:  tensor(0.0050)
epoch:  411	arg

epoch:  441	argmax time taken,  0.42049717903137207
assembling the matrix time taken:  0.00025844573974609375
solving Ax = b time taken:  0.03951740264892578
		 time taken minimize linear layer:  0.039811134338378906
current error:  tensor(0.0054)
epoch:  442	argmax time taken,  0.4208216667175293
assembling the matrix time taken:  0.0002510547637939453
solving Ax = b time taken:  0.0402524471282959
		 time taken minimize linear layer:  0.04053807258605957
current error:  tensor(0.0047)
epoch:  443	argmax time taken,  0.42055797576904297
assembling the matrix time taken:  0.00025272369384765625
solving Ax = b time taken:  0.03961038589477539
		 time taken minimize linear layer:  0.039897918701171875
current error:  tensor(0.0047)
epoch:  444	argmax time taken,  0.4205596446990967
assembling the matrix time taken:  0.0002560615539550781
solving Ax = b time taken:  0.04024004936218262
		 time taken minimize linear layer:  0.04053068161010742
current error:  tensor(0.0048)
epoch:  445	arg

epoch:  475	argmax time taken,  0.42179369926452637
assembling the matrix time taken:  0.0002536773681640625
solving Ax = b time taken:  0.046579599380493164
		 time taken minimize linear layer:  0.04686856269836426
current error:  tensor(0.0048)
epoch:  476	argmax time taken,  0.4211752414703369
assembling the matrix time taken:  0.00026226043701171875
solving Ax = b time taken:  0.04629063606262207
		 time taken minimize linear layer:  0.04658699035644531
current error:  tensor(0.0049)
epoch:  477	argmax time taken,  0.4229741096496582
assembling the matrix time taken:  0.0002498626708984375
solving Ax = b time taken:  0.04682040214538574
		 time taken minimize linear layer:  0.047104835510253906
current error:  tensor(0.0053)
epoch:  478	argmax time taken,  0.42145729064941406
assembling the matrix time taken:  0.0002598762512207031
solving Ax = b time taken:  0.0469059944152832
		 time taken minimize linear layer:  0.04720139503479004
current error:  tensor(0.0048)
epoch:  479	argm

epoch:  509	argmax time taken,  0.4251558780670166
assembling the matrix time taken:  0.00024390220642089844
solving Ax = b time taken:  0.050681114196777344
		 time taken minimize linear layer:  0.05096006393432617
current error:  tensor(0.0047)
epoch:  510	argmax time taken,  0.42212510108947754
assembling the matrix time taken:  0.00025010108947753906
solving Ax = b time taken:  0.05099344253540039
		 time taken minimize linear layer:  0.051278114318847656
current error:  tensor(0.0055)
epoch:  511	argmax time taken,  0.42195987701416016
assembling the matrix time taken:  0.00025177001953125
solving Ax = b time taken:  0.051637887954711914
		 time taken minimize linear layer:  0.05193042755126953
current error:  tensor(0.0047)
epoch:  512	argmax time taken,  0.4253995418548584
assembling the matrix time taken:  0.0002491474151611328
solving Ax = b time taken:  0.05810356140136719
		 time taken minimize linear layer:  0.05838727951049805
current error:  tensor(0.0048)
time taken:  23

## Random dictionary 

In [17]:
if __name__ == "__main__": 
    def target(x):
        return torch.sin(pi*x[:,0:1])*torch.sin(pi*x[:,1:2])*torch.sin(pi*x[:,2:3]) *torch.sin(pi*x[:,3:4])

    experiment_label = "ex1"
    for k in [4,1]: 
        s = 1 
        for N0 in [2**6, 2**7, 2**8]: 
            print()
            print() 
            exponent = 9  
            num_epochs=  2**exponent 
            M = 2**19 # around 50w 
            print(M)
            my_model = None 
            
            err, my_model = OGAL2FittingReLU4D_QMC(my_model,target, \
                        s,N0,num_epochs, M, k = k, linear_solver = "direct", num_batches = 1)
            

            filename = experiment_label + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(k,s * N0,num_epochs)
            torch.save(err,filename)
            filename = experiment_label + "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(k,s * N0,num_epochs)
            torch.save(my_model.state_dict(),filename) 

            print_convergence_order(err,exponent+1) 




524288
generate sob sequence: 0.011663198471069336
using linear solver:  direct
epoch:  1	argmax time taken,  0.0037081241607666016
assembling the matrix time taken:  0.0003464221954345703
solving Ax = b time taken:  0.00027370452880859375
		 time taken minimize linear layer:  0.0006606578826904297
current error:  tensor(0.2080)
epoch:  2	argmax time taken,  0.0037636756896972656
assembling the matrix time taken:  0.00033092498779296875
solving Ax = b time taken:  0.0009398460388183594
		 time taken minimize linear layer:  0.0013103485107421875
current error:  tensor(0.2005)
epoch:  3	argmax time taken,  0.004207611083984375
assembling the matrix time taken:  0.0003161430358886719
solving Ax = b time taken:  0.0009596347808837891
		 time taken minimize linear layer:  0.0013120174407958984
current error:  tensor(0.1961)
epoch:  4	argmax time taken,  0.00427699089050293
assembling the matrix time taken:  0.0002875328063964844
solving Ax = b time taken:  0.0010471343994140625
		 time ta

solving Ax = b time taken:  0.0037550926208496094
		 time taken minimize linear layer:  0.004281520843505859
current error:  tensor(0.0664)
epoch:  45	argmax time taken,  0.007972240447998047
assembling the matrix time taken:  0.0003216266632080078
solving Ax = b time taken:  0.003947734832763672
		 time taken minimize linear layer:  0.004307270050048828
current error:  tensor(0.0593)
epoch:  46	argmax time taken,  0.008002519607543945
assembling the matrix time taken:  0.00029397010803222656
solving Ax = b time taken:  0.003938436508178711
		 time taken minimize linear layer:  0.004267454147338867
current error:  tensor(0.0585)
epoch:  47	argmax time taken,  0.008057832717895508
assembling the matrix time taken:  0.00028514862060546875
solving Ax = b time taken:  0.004789590835571289
		 time taken minimize linear layer:  0.005110263824462891
current error:  tensor(0.0529)
epoch:  48	argmax time taken,  0.00810098648071289
assembling the matrix time taken:  0.0002903938293457031
solvin

solving Ax = b time taken:  0.007608175277709961
		 time taken minimize linear layer:  0.00796818733215332
current error:  tensor(0.0363)
epoch:  81	argmax time taken,  0.01067042350769043
assembling the matrix time taken:  0.00029468536376953125
solving Ax = b time taken:  0.0081329345703125
		 time taken minimize linear layer:  0.008463382720947266
current error:  tensor(0.0359)
epoch:  82	argmax time taken,  0.011099576950073242
assembling the matrix time taken:  0.0002808570861816406
solving Ax = b time taken:  0.008159160614013672
		 time taken minimize linear layer:  0.008475065231323242
current error:  tensor(0.0354)
epoch:  83	argmax time taken,  0.011119604110717773
assembling the matrix time taken:  0.0002880096435546875
solving Ax = b time taken:  0.008270263671875
		 time taken minimize linear layer:  0.008592844009399414
current error:  tensor(0.0350)
epoch:  84	argmax time taken,  0.009335041046142578
assembling the matrix time taken:  0.0002808570861816406
solving Ax = b

solving Ax = b time taken:  0.01036834716796875
		 time taken minimize linear layer:  0.010721445083618164
current error:  tensor(0.0232)
epoch:  119	argmax time taken,  0.01059103012084961
assembling the matrix time taken:  0.0002803802490234375
solving Ax = b time taken:  0.010497570037841797
		 time taken minimize linear layer:  0.010811805725097656
current error:  tensor(0.0231)
epoch:  120	argmax time taken,  0.010679483413696289
assembling the matrix time taken:  0.00028252601623535156
solving Ax = b time taken:  0.010338306427001953
		 time taken minimize linear layer:  0.010656595230102539
current error:  tensor(0.0229)
epoch:  121	argmax time taken,  0.013516664505004883
assembling the matrix time taken:  0.000286102294921875
solving Ax = b time taken:  0.010621309280395508
		 time taken minimize linear layer:  0.010942459106445312
current error:  tensor(0.0225)
epoch:  122	argmax time taken,  0.010638236999511719
assembling the matrix time taken:  0.00028514862060546875
solvi

current error:  tensor(0.0140)
epoch:  155	argmax time taken,  0.014296531677246094
assembling the matrix time taken:  0.0003044605255126953
solving Ax = b time taken:  0.016004323959350586
		 time taken minimize linear layer:  0.016346216201782227
current error:  tensor(0.0139)
epoch:  156	argmax time taken,  0.0143585205078125
assembling the matrix time taken:  0.0003037452697753906
solving Ax = b time taken:  0.015654802322387695
		 time taken minimize linear layer:  0.015995502471923828
current error:  tensor(0.0136)
epoch:  157	argmax time taken,  0.014374017715454102
assembling the matrix time taken:  0.00030040740966796875
solving Ax = b time taken:  0.016126632690429688
		 time taken minimize linear layer:  0.016463756561279297
current error:  tensor(0.0134)
epoch:  158	argmax time taken,  0.014789342880249023
assembling the matrix time taken:  0.00026869773864746094
solving Ax = b time taken:  0.016097307205200195
		 time taken minimize linear layer:  0.016401290893554688
curr

...

current error:  tensor(0.0092)
epoch:  357	argmax time taken,  0.02879476547241211
assembling the matrix time taken:  0.0003180503845214844
solving Ax = b time taken:  0.04275226593017578
		 time taken minimize linear layer:  0.0431365966796875
current error:  tensor(0.0092)
epoch:  358	argmax time taken,  0.028760433197021484
assembling the matrix time taken:  0.00033736228942871094
solving Ax = b time taken:  0.04251694679260254
		 time taken minimize linear layer:  0.042900800704956055
current error:  tensor(0.0092)
epoch:  359	argmax time taken,  0.02889728546142578
assembling the matrix time taken:  0.0003204345703125
solving Ax = b time taken:  0.04286694526672363
		 time taken minimize linear layer:  0.0432276725769043
current error:  tensor(0.0092)
epoch:  360	argmax time taken,  0.028951406478881836
assembling the matrix time taken:  0.00038886070251464844
solving Ax = b time taken:  0.04191780090332031
		 time taken minimize linear layer:  0.0423588752746582
current error:  t

current error:  tensor(0.0085)
epoch:  391	argmax time taken,  0.02985239028930664
assembling the matrix time taken:  0.0003314018249511719
solving Ax = b time taken:  0.05629134178161621
		 time taken minimize linear layer:  0.05666232109069824
current error:  tensor(0.0085)
epoch:  392	argmax time taken,  0.029896020889282227
assembling the matrix time taken:  0.00029659271240234375
solving Ax = b time taken:  0.05296492576599121
		 time taken minimize linear layer:  0.0532989501953125
current error:  tensor(0.0084)
epoch:  393	argmax time taken,  0.0299224853515625
assembling the matrix time taken:  0.0003192424774169922
solving Ax = b time taken:  0.055840253829956055
		 time taken minimize linear layer:  0.05621695518493652
current error:  tensor(0.0084)
epoch:  394	argmax time taken,  0.02991318702697754
assembling the matrix time taken:  0.0004169940948486328
solving Ax = b time taken:  0.0549619197845459
		 time taken minimize linear layer:  0.05564260482788086
current error:  

solving Ax = b time taken:  0.05342221260070801
		 time taken minimize linear layer:  0.05381274223327637
current error:  tensor(0.0079)
epoch:  425	argmax time taken,  0.030823945999145508
assembling the matrix time taken:  0.00031948089599609375
solving Ax = b time taken:  0.05524444580078125
		 time taken minimize linear layer:  0.05562853813171387
current error:  tensor(0.0078)
epoch:  426	argmax time taken,  0.030894994735717773
assembling the matrix time taken:  0.00033545494079589844
solving Ax = b time taken:  0.054718971252441406
		 time taken minimize linear layer:  0.05512547492980957
current error:  tensor(0.0078)
epoch:  427	argmax time taken,  0.031003475189208984
assembling the matrix time taken:  0.0003292560577392578
solving Ax = b time taken:  0.05547332763671875
		 time taken minimize linear layer:  0.05584383010864258
current error:  tensor(0.0078)
epoch:  428	argmax time taken,  0.031145334243774414
assembling the matrix time taken:  0.0002944469451904297
solving A

solving Ax = b time taken:  0.06406140327453613
		 time taken minimize linear layer:  0.06453561782836914
current error:  tensor(0.0072)
epoch:  459	argmax time taken,  0.0319514274597168
assembling the matrix time taken:  0.0003218650817871094
solving Ax = b time taken:  0.06446552276611328
		 time taken minimize linear layer:  0.06485152244567871
current error:  tensor(0.0072)
epoch:  460	argmax time taken,  0.03186798095703125
assembling the matrix time taken:  0.0003864765167236328
solving Ax = b time taken:  0.06348633766174316
		 time taken minimize linear layer:  0.06406927108764648
current error:  tensor(0.0072)
epoch:  461	argmax time taken,  0.031885623931884766
assembling the matrix time taken:  0.0003249645233154297
solving Ax = b time taken:  0.06447577476501465
		 time taken minimize linear layer:  0.06484770774841309
current error:  tensor(0.0072)
epoch:  462	argmax time taken,  0.03201103210449219
assembling the matrix time taken:  0.000331878662109375
solving Ax = b ti

solving Ax = b time taken:  0.06537437438964844
		 time taken minimize linear layer:  0.06580400466918945
current error:  tensor(0.0068)
epoch:  493	argmax time taken,  0.03296351432800293
assembling the matrix time taken:  0.0003299713134765625
solving Ax = b time taken:  0.0667569637298584
		 time taken minimize linear layer:  0.06712603569030762
current error:  tensor(0.0068)
epoch:  494	argmax time taken,  0.033040761947631836
assembling the matrix time taken:  0.00040030479431152344
solving Ax = b time taken:  0.06615257263183594
		 time taken minimize linear layer:  0.06665754318237305
current error:  tensor(0.0068)
epoch:  495	argmax time taken,  0.03299236297607422
assembling the matrix time taken:  0.00042057037353515625
solving Ax = b time taken:  0.0667576789855957
		 time taken minimize linear layer:  0.06725740432739258
current error:  tensor(0.0068)
epoch:  496	argmax time taken,  0.03305864334106445
assembling the matrix time taken:  0.00032448768615722656
solving Ax = b